In [1]:
#!/usr/bin/env python
import numpy as np
import math as m
import pylab as pl
from matplotlib import colors
from matplotlib import gridspec
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter
from colormaps import spectral
from matplotlib.colors import ColorConverter

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# Font properties
# Note: TNR required for bolding due to font manager bug
# Math font for compact, times-like scientific notation
# Exit out of notebook before changing
times_font=True
if times_font:
    font_bold = fm.FontProperties(family='Times New Roman', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Times', style='normal', size=14, weight='normal', stretch='normal')
    font_name ='Times'
    plt.rcParams["mathtext.fontset"] = "stix"#"stixsans"
else:
    font_name ='Dejavu Sans'
    font_bold = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='normal', stretch='normal')


In [3]:
# Spectral custom colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

# Data file names
#data_dir   = '/Users/nicholebarry/MWA/data/Woden_tests/'
#data_dir = '/Users/nicholebarry/MWA/data/bubble_simulations/WODEN/mod/'
data_dir = '/Users/z3543600/tmp/w_plane/'
#data_base  = ['1061316296_gridded_uvf__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_no_horizon_wedge_kperplambda10-50_1dkpower.txt']
#data_base  = ['theory_z7_power_1d_check.txt',
#              '1090008640_gridded_uvf__even_odd_joint_fullimg_dirty_xx_dft_averemove_swbh_dencorr_kperplambda10-50_1dkpower.txt',
#              '1061316296_gridded_uvf__even_odd_joint_fullimg_dirty_xx_dft_averemove_swbh_dencorr_kperplambda1-100_1dkpower_FHD.txt']
data_base = ['1125935680_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_kperplambda10-50_1dkpower.txt',
            '1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_kperplambda10-50_1dkpower.txt',
            'Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_kperplambda10-50_1dkpower.txt',
            'w_test_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_kperplambda10-50_1dkpower.txt']
data_col = [[0,1],[0,1],[0,1],[0,1]]

# Plot styles per data set
colors=['j','i','h','g','e']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

for file_i, file_name in enumerate(data_base):
    #
    # Load data and select
    data = np.loadtxt(data_dir+file_name,skiprows=1)
    k_edges = data[:,(data_col[file_i])[0]] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,(data_col[file_i])[1]] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = (power)*(k_center**3.)/(2*np.pi**2)
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
#     if file_i==0:
#         k_edges = k_edges*1.4
#     if file_i==1:
#         delta = delta / 4. #Stokes I oopsie
#     if file_i==2:
#         delta = delta /2. #why though
    k_edges_hist  = np.repeat(k_edges,2)[1:-1]
    delta_hist  = np.repeat(delta,2)
    #if file_i == 1:
    #    delta_hist *= 25.
        
    # Plot the data on the initialised figure
    pl.plot(k_edges_hist, delta_hist, ls='solid', color=colors[file_i],linewidth=2)
        
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(1e-2,2)
pl.ylim(1e-1, 1e8)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
#pl.yticks([1e-0,1e3,1e6],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Legend
#f1_label = ['EoR z=7 theory','WODEN vis through FHD/eppsilon', 'FHD simulation']
f1_label = ['Model: Quadratic beam\ninterpolation','Model: Default beam\ninterpolation']
patches = [ pl.plot([],[], mec=None, color=colors[type_i], 
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.79, 0.71),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
#legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
a.add_artist(legend1)


pl.savefig(data_dir+'Power_Spectrum_Plot.png', bbox_inches='tight', dpi=128)
pl.close('all')


In [32]:
#EoR plotting only

# Spectral custom colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

# Data file names
#data_dir   = '/Users/nicholebarry/MWA/data/bubble_simulations/'
data_dir   = '/Users/z3543600/MWA/data/w_plane_tests/'
# data_base = ['no_w/1125941048_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
#              'no_w/Combined_obs_1125941048_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
#              'no_w/Combined_obs_EoRlast3_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
#              'no_w/Combined_obs_EoRfirst3_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
#              'no_w/Combined_obs_EoR_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt']
# f1_label = ['+1 obs, grid','+1 obs, Healpix','Last 3, Healpix','First 3, Healpix','All 4']
# data_base = ['noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg2_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg3_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg4_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg5_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg6_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg7_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg8_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg9_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt',
#             'noise_uv_avg/1125939344_gridded_uvf_XX__even_odd_joint_noimgclip_uvavg10_dirty_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower_notfixed.txt']
#data_base = ['no_w/Combined_obs_1125941048_cubeXX__even_odd_joint_noimgclip_res_xx_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
data_base = ['eor_compare/now/Combined_obs_lst_matched_cubeXX__even_odd_joint_noimgclip_res_yy_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
             'eor_compare/now/lst_matched_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_yy_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
             'eor_compare/w/Combined_obs_lst_matched_cubeXX__even_odd_joint_noimgclip_res_yy_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
             'eor_compare/w/lst_matched_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_yy_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt',
             'eor_compare/aw/lst_matched_gridded_uvf_XX__even_odd_joint_noimgclip_dirty_yy_dft_averemove_swbh_dencorr_logk_kperplambda10-50_1dkpower.txt']

#f1_label = ['0.5 $\lambda$/pix','avg 2','avg 3','avg 4','avg 5','avg 6','avg 7','avg 8','avg 9','avg 10']
f1_label = ['HEALPix, no w','Gridded, no w','HEALPix, w-proj','Gridded, w-proj','Gridded, aw-proj']

data_col = [[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]

# Plot styles per data set
all_colors=['j','i','g','e','h','brown','black']
all_linestyles=['-','-','-','-','-','-','--']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

brad_data = np.loadtxt(data_dir+'delTps_estimate_DIM6400_NSlices420_7500Mpc_lighttravel.txt')
   
pl.plot(brad_data[:,0]/0.68, brad_data[:,1],
        linestyle='-', color='black', label='Expected', alpha=0.3)

for file_i, file_name in enumerate(data_base):
    #
    # Load data and select
    data = np.loadtxt(data_dir+file_name,skiprows=1)
    k_edges = data[:,(data_col[file_i])[0]] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,(data_col[file_i])[1]] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Construct power in mK^2 units
    delta = (power)*(k_center**3.)/(2*np.pi**2) #/ 2 * 1.0
    # if file_i != 0:
    #     power *= independent_vis[file_i] 
    # if file_i == 0:
    #     power *= 5/6
    # # if file_i == 1:
    # #     power *= 5/6
    # if file_i == 2:
    #     power *= 5/6
    # if file_i == 3:
    #     power *= 5/6
    # if file_i == 4:
    #     power *= 5/6
    # if file_i == 6:
    #     power *= 5/6
    # if file_i == 7:
    #     power *= 5/6
    # if file_i == 8:
    #     power *= 5/6
    # if file_i == 2:
    #     power *=2
    # if file_i == 6:
    #      power[:] = 8.923
    
    #delta = ((power)) #/ 8.923 #/ 2 * 2.651
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1]
    delta_hist  = np.repeat(delta,2)

        
    # Plot the data on the initialised figure

    pl.plot(k_edges_hist, delta_hist, color=all_colors[file_i], linestyle=all_linestyles[file_i],linewidth=2)
        
# Final plotting settings and save

#pl.loglog()

pl.xscale('log')
pl.yscale('log')

pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(1e-2,2)
pl.ylim(1e-2, 100)
#pl.ylim(1e0, 3e1)
#pl.ylim(-1e2, 1e2)
pl.ylabel(r'$\Delta^2$ (mK$^2$)', fontsize=fontsize, family=font_name)
#pl.ylabel(r'Power (mK$^2$ h$^{-3}$ Mpc$^3$)', fontsize=fontsize, family=font_name)
#pl.ylabel(r'Ratio Power', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)
pl.title('Testing w-corrections using EoR simulation from Line et al. 2024') #with Line et. al 2024 density corr')

# Tick formatting
#pl.yticks([1e-2,2e0],fontname = font_name)
#pl.yticks([-5e1,0,5e1],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Legend
#f1_label = ['-2 pointing obs, grid', '-1 pointing obs, grid','0 pointing obs, grid', '1 pointing obs, grid', 'integrated, HEALPix cubes','integrated, grid']
#f1_label = ['-2 obs','-1 obs','0 obs','1 obs','All 4']

patches = [ pl.plot([],[], mec=None, color=all_colors[type_i], linestyle=all_linestyles[type_i],
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]

legend2 = pl.legend(handles=patches, bbox_to_anchor=(0.76, 0.25),loc='center', ncol=1, frameon=False, 
                    fontsize=legendsize,title_fontsize=11,prop=font, title="15 obs, YY")
#legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
#legend1._legend_box.align='left'
legend2._legend_box.align='left'
a.add_artist(legend2)


pl.savefig(data_dir+'eor_compare_YY.pdf', bbox_inches='tight', dpi=128)
pl.close('all')

In [75]:
#k0 plotting

# Spectral custom colors
ColorConverter.colors['e']=(94/256, 79/256, 162/256)
ColorConverter.colors['g']=(50/256, 136/256, 189/256)
ColorConverter.colors['h']=(102/256, 194/256, 165/256)
ColorConverter.colors['i']=(253/256, 174/256, 97/256)
ColorConverter.colors['j']=(219/256, 55/256, 82/256)

data_dir = '/Users/nicholebarry/MWA/data/k0_LoBES_compare/'
data_base  = ['1088285600_gridded_uvf__even_odd_joint_fullimg_dirty_yy_dft_averemove_swbh_dencorr_k0power.txt',
              '1088285600_gridded_uvf__even_odd_joint_fullimg_res_yy_dft_averemove_swbh_dencorr_k0power.txt',
              '1088285600_gridded_uvf__even_odd_joint_fullimg_dirty_yy_dft_averemove_swbh_dencorr_k0power_LoBES_extraremoved.txt',
              '1088285600_gridded_uvf__even_odd_joint_fullimg_res_yy_dft_averemove_swbh_dencorr_k0power_LoBES_extraremoved.txt']
data_col = [[0,1],[0,1],[0,1],[0,1]]

# Plot styles per data set
colors=['j','i','h','g','e']

fontsize=16
ticksize=16
legendsize=16

# Initialise the plot
pl.figure(figsize=(8,6))

for file_i, file_name in enumerate(data_base):
    #
    # Load data and select
    data = np.loadtxt(data_dir+file_name,skiprows=1)
    k_edges = data[:,(data_col[file_i])[0]] # Lower and upper edge of bin, (h Mpc^-1)
    power = data[1:,(data_col[file_i])[1]] # Observed power in each bin, (mK^2 h^-3 Mpc^3), first entry is a placeholder
        
    # Construct k center from the edges
    n_k_edges = len(k_edges)
    k_center = (k_edges[1:n_k_edges]+k_edges[0:n_k_edges-1])/2.
        
    # Create histogram style data set by repeating the data 
    # Duplicate the bin edges so each bin has left and right data point
    # Duplicate the power so each bin edge has a measurement
    k_edges_hist  = np.repeat(k_edges,2)[1:-1]
    power_hist  = np.repeat(power,2)
    #if file_i == 1:
    #    delta_hist *= 25.
        
    # Plot the data on the initialised figure
    #if (file_i == 1) or (file_i == 3):
    #    pl.plot(k_edges_hist, power_hist/power_comp, ls='solid', color=colors[file_i],linewidth=2)
    #if (file_i == 0) or (file_i == 2):
    #    power_comp = power_hist
    pl.plot(k_edges_hist, power_hist, ls='solid', color=colors[file_i],linewidth=2)
# Final plotting settings and save

pl.loglog()
pl.tick_params(direction='in', color='k', which='both', top=True, bottom=True, left=True, right=True, labelsize=ticksize)
pl.xlim(1e-2,1e-1)
pl.ylim(5e11, 2e15)
#pl.ylim(1e-2, 1e0)
pl.ylabel(r'k0 power (mK$^2$ h$^{-3}$ Mpc$^3$)', fontsize=fontsize, family=font_name)
pl.xlabel(r'$k_{\perp}$ (h Mpc$^{-1}$)', fontsize=fontsize, family=font_name)

# Tick formatting
#pl.yticks([1e-0,1e3,1e6],fontname = font_name)
pl.xticks(fontname = font_name)
a=pl.gca()
a.yaxis.set_minor_locator(ticker.LogLocator(base=10, numticks=20))
a.yaxis.set_minor_formatter(ticker.NullFormatter())

# Legend
f1_label = ['Calibrated data, GLEAM', 'Residual data, GLEAM', 'Calibrated data, LoBES', 'Residual data, LoBES']
#f1_label = ['Ratio of input to remaining power, GLEAM', 'Ratio of input to remaining power, LoBES']
patches = [ pl.plot([],[], mec=None, color=colors[type_i], 
            label="{:s}".format(f1_label[type_i]) )[0]  for type_i in range(len(f1_label)) ]
legend1 = pl.legend(handles=patches, bbox_to_anchor=(0.22, 0.15),loc='center', ncol=1, frameon=False,
                    fontsize=legendsize,title_fontsize=legendsize,prop=font)
#legend1.set_title(title="Derivative matched\nto control cut percent", prop=font_bold)
legend1._legend_box.align='left'
a.add_artist(legend1)


pl.savefig(data_dir+'Power_Spectrum_Plot.pdf', bbox_inches='tight', dpi=128)
pl.close('all')



FileNotFoundError: /Users/nicholebarry/MWA/data/k0_LoBES_compare/1088285600_gridded_uvf__even_odd_joint_fullimg_dirty_yy_dft_averemove_swbh_dencorr_k0power.txt not found.

<Figure size 800x600 with 0 Axes>